In [1]:
using Pkg
Pkg.activate("C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\dynamical-systems\\env\\integrate\\")

using StaticArrays, DifferentialEquations, DynamicalSystems, JLD

  Activating project at `C:\Users\Alex\Desktop\repo\2612\0903\dynamical-systems\env\integrate`


In [91]:
@inbounds U(y, p) = p[8] + p[9] / ( 1.0 + exp( -50.0 * (y - p[7]) ) )
@inbounds σ(x, p) = 1.0 / ( 1.0 + exp( -20.0 * (x-p[6]) ) )
@inbounds g(E, x, y, p, U_) = log( 1.0 + exp( (p[5] * U_ * x * E + p[11]  ) / (p[1]) ) )

@inbounds function TM(u, p, t)
    
    U_ = U(u[3], p)
    
    du1 = (-u[1] + p[1] * g(u[1], u[2], u[3], p, U_) ) / p[2]
    du2 = (1.0 - u[2]) / p[3] - U_*u[2]*u[1]
    du3 = (-u[3])/p[4] + p[10] * σ(u[2], p)
    
    return SVector(du1, du2, du3)
end
@inbounds function jacob_TM_(u, p, t)
    
    U(y, p, exp50) = p[8] + p[9] / ( 1.0 + exp50 )
    U_y(y, p, exp50) = (50.0 * p[9] * exp50) / (1.0 + exp50)^2
    g(E, x, y, p, U_) = exp((p[5]  * U_ * x * E + p[11]) / p[1])
    σ_der(x, p) = exp( (-20.0) * (x - p[6]) )
    exp50 = exp(-50.0 * (u[3] - p[7]))
    
    U_ = U(u[3], p, exp50)
    Uy = U_y(u[3], p, exp50)
    g_ = g(u[1], u[2], u[3], p, U_)
    σ_deri = σ_der(u[2], p)
    
    g_plus = 1.0 + g_
    g_mult = g_ * U_
    g_plus_mult = p[2] * (g_plus)
    u1p5 = p[5] * u[1]
    Uyu2 = Uy * u[2]
    
    E_E = (-1.0 + ((J * u[2] * g_mult)) / (g_plus) ) / p[2]
    E_x = (u1p5 * g_mult) / (g_plus_mult)
    E_y = (u1p5 * Uyu2 * g_) / (g_plus_mult)
    
    x_E = -U_ * u[2]
    x_x = -1.0 / p[3] - U_ * u[1]
    x_y = -Uyu2 * u[1]
    
    y_x = 20.0 * p[10] * σ_deri / (1.0 + σ_deri)^2
    y_y = -1.0/p[4]
    
    SMatrix{3,3}(E_E, x_E, 0.0,
        E_x, x_x, y_x,
        E_y, x_y, y_y)
end;

In [92]:
get_step(pstart, pend, length) = (abs(pstart)-abs(pend)) / length

get_step (generic function with 1 method)

In [93]:
function outputPx(idxPx, Px, u0)
    println("Индекс $Pxlabel: $idxPx; $Pxlabel: $Px");flush(stdout)
    println("Начальные условия: $u0");flush(stdout)
end
function outputPy(idxPx, Px, u0)
    println("Индекс $Pylabel: $idxPx; $Pylabel: $Px");flush(stdout)
    println("Начальные условия: $u0");flush(stdout)
end
function output(idx_Px, idx_Py, Px, Py, u0)
    println("Индекс $Pxlabel: $idx_Px; $Pxlabel: $Px");flush(stdout)
    println("Индекс $Pylabel: $idx_Py; $Pylabel: $Py");flush(stdout)
    println("Начальные условия: $u0");flush(stdout)
end
function outputIter(ΛΛ, u0)
    println("Спектр: $ΛΛ");flush(stdout)
    println("Последняя точка: $u0");flush(stdout)
end
function separate()
    println();flush(stdout)
    println("---------");flush(stdout)
end;

In [94]:
function save_output(idx_Px, idx_Py, Px, Py, ΛΛ, u0)
    open(file_LS, "a") do io
            println(io, " $idx_Px $idx_Py $Px $Py $(ΛΛ[1]) $(ΛΛ[2]) $(ΛΛ[3])")
    end
    open(file_u0s, "a") do io
            println(io, " $idx_Px $idx_Py $Px $Py $(u0[1]) $(u0[2]) $(u0[3])")
    end
end

save_output (generic function with 1 method)

In [95]:
function init_ds_Px(p, Px, Pxindex, u0)
    p[Pxindex] = Px; p[Pyindex] = Py_start
    ds = CoupledODEs(TM, u0, p, diffeq = integ_set)
    return ds
end
function init_ds_Py(p, Py, Pyindex, u0)
    p[Pyindex] = Py; p[Pxindex] = Px_start
    ds = CoupledODEs(TM, u0, p, diffeq = integ_set)
    return ds
end
function init_ds(p, Pxindex, Pyindex, Px, Py, u0)
    p[Pyindex] = Py; p[Pxindex] = Px
    ds = CoupledODEs(TM, u0, p,  diffeq = integ_set)
    return ds
end
function goto_attractor(ds, t, tt)
    tr, _ = trajectory(ds, t; Ttr = tt)
    u0 = tr[end]
    return u0
end
function spectrum(ds)
     ΛΛ = lyapunovspectrum(ds, t)
end

spectrum (generic function with 1 method)

In [96]:
Pxindex = 11; Pyindex = 8 # Индекс параметра по оси х и оси у
Pxlabel = "I0"; Pylabel = "U0"; # Обозначение параметров. Нужно для вывода.

In [97]:
"""
t -- Время в течении которого вычисляется спектр
tt -- Время которое отбрасывается
tstep -- Шаг по времени для интегратора
time_attract -- Время в течении которого считается траектория

integ_set -- Tuple для настроек интегратора
"""
t = 1000; tt = 500; tstep = 0.001
time_attract = 100.0

integ_set = (alg = RK4(), adaptive = false, dt = tstep);

In [98]:
const τ = 0.013;  const τD = 0.080;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58;  U0 = 0.22;  const ΔU0 = 0.305; I0 = -0.5

p = [α, τ, τD, τy, J, xthr, ythr, 0, ΔU0, β, 0]

u0 = [8.39353, 0.73803, 0.429055]

3-element Vector{Float64}:
 8.39353
 0.73803
 0.429055

In [99]:
Px_start = -0.5; Px_end = -2.0; default_length_range_x = 300
Py_start = 0.22; Py_end = 0.55; default_length_range_y = 300

step_x = get_step(Px_start, Px_end, default_length_range_x)
step_y = get_step(Py_start, Py_end, default_length_range_y);

In [100]:
file_LS = "Test_LS.txt"
file_u0s = "Test_u0s.txt"

"Test_u0s.txt"

In [ ]:
Px = Px_start
Py = Py_start
idx_Px = 0
idx_Py = 0

while (Py <= Py_end)
    idx_Py+=1
    
    if idx_Py == 1
        global u0_lc = u0
    end
    
    outputPy(idx_Py ,Py, u0_lc)
    
    ds = init_ds_Py(p, Py, Pyindex, u0_lc) # Инициализация системы
    u0_lc = goto_attractor(ds, time_attract, tt) # Выход на аттрактор
    ds = init_ds_Py(p, Py, Pyindex, u0_lc)
    ΛΛ = spectrum(ds) # Вычисление спектра
    
    outputIter(ΛΛ, u0_lc)
    
    save_output(idx_Px, idx_Py, Px, Py, ΛΛ, u0_lc) # Сохранение значений
    
    separate()
    
    Py+=step_y
    
end

idx_Px  -=1
idx_py -=1;

Индекс U0: 1; U0: 0.22
Начальные условия: [8.39353, 0.73803, 0.429055]
Спектр: [-3.1465213850101486, -3.1456245098847786, -7.198524508894548]
Последняя точка: [9.61839650326879, 0.7364220338699033, 0.42819908049470906]

---------
Индекс U0: 2; U0: 0.2189
Начальные условия: [9.61839650326879, 0.7364220338699033, 0.42819908049470906]
Спектр: [-3.1837986045266264, -3.183243701032597, -7.120582064185706]
Последняя точка: [9.614932378192913, 0.7365125099148341, 0.42863883418854626]

---------
Индекс U0: 3; U0: 0.21780000000000002
Начальные условия: [9.614932378192913, 0.7365125099148341, 0.42863883418854626]
Спектр: [-3.221425578870244, -3.2219138862684664, -7.041200313280392]
Последняя точка: [9.61142375920964, 0.7366041485132035, 0.429084347162542]

---------
Индекс U0: 4; U0: 0.21670000000000003
Начальные условия: [9.61142375920964, 0.7366041485132035, 0.429084347162542]
Спектр: [-3.2610223150734017, -3.2600286405989376, -6.9603642243860655]
Последняя точка: [9.607868970698279, 0.7366969

Индекс U0: 31; U0: 0.1870000000000003
Начальные условия: [9.490599538609242, 0.7397601433549332, 0.44448912966543985]
Спектр: [-4.205236093480463, -4.584927760900748, -4.583270707615641]
Последняя точка: [9.48459748475526, 0.739916935451477, 0.4452572913766421]

---------
Индекс U0: 32; U0: 0.18590000000000031
Начальные условия: [9.48459748475526, 0.739916935451477, 0.4452572913766421]
Спектр: [-4.082977707717442, -4.643428436365219, -4.6416362583446205]
Последняя точка: [9.478419356470177, 0.7400783286947933, 0.446048248024621]

---------
Индекс U0: 33; U0: 0.18480000000000033
Начальные условия: [9.478419356470177, 0.7400783286947933, 0.446048248024621]
Спектр: [-3.959583467876332, -4.701852861616801, -4.701051903897518]
Последняя точка: [9.472053293508573, 0.74024463305862, 0.44686353723460304]

---------
Индекс U0: 34; U0: 0.18370000000000034
Начальные условия: [9.472053293508573, 0.74024463305862, 0.44686353723460304]
Спектр: [-3.835116364324064, -4.761341337785925, -4.760303168966


---------
Индекс U0: 61; U0: 0.1540000000000006
Начальные условия: [9.083319156419915, 0.7504032059084558, 0.4969958584308886]
Спектр: [-0.5764008876251304, -6.209071817210036, -6.2073588181709365]
Последняя точка: [9.042357145893577, 0.7514740764511437, 0.5022961499801537]

---------
Индекс U0: 62; U0: 0.15290000000000062
Начальные условия: [9.042357145893577, 0.7514740764511437, 0.5022961499801537]
Спектр: [-0.5075402533233838, -6.2227684647686505, -6.223000469070998]
Последняя точка: [8.995605639775908, 0.7526964133215495, 0.5083440121124884]

---------
Индекс U0: 63; U0: 0.15180000000000063
Начальные условия: [8.995605639775908, 0.7526964133215495, 0.5083440121124884]
Спектр: [-0.4504810674771663, -6.230181250549895, -6.227895393326056]
Последняя точка: [8.942445994632301, 0.7540864422855104, 0.5152166373043597]

---------
Индекс U0: 64; U0: 0.15070000000000064
Начальные условия: [8.942445994632301, 0.7540864422855104, 0.5152166373043597]
Спектр: [-0.4056423986973004, -6.226131621

Последняя точка: [6.499734429378252, 0.818275077091513, 0.788685863755846]

---------
Индекс U0: 91; U0: 0.12100000000000086
Начальные условия: [6.499734429378252, 0.818275077091513, 0.788685863755846]
Спектр: [-0.30917087870715404, -5.492347341913895, -5.488830901657253]
Последняя точка: [6.393251976873016, 0.821097592232177, 0.7975877795044468]

---------
Индекс U0: 92; U0: 0.11990000000000085
Начальные условия: [6.393251976873016, 0.821097592232177, 0.7975877795044468]
Спектр: [-0.30918110005302224, -5.47882342016899, -5.479790135869874]
Последняя точка: [6.285784306239056, 0.8239496591121519, 0.8062762540799673]

---------
Индекс U0: 93; U0: 0.11880000000000085
Начальные условия: [6.285784306239056, 0.8239496591121519, 0.8062762540799673]
Спектр: [-0.30919256612212886, -5.470759015015095, -5.472075281012849]
Последняя точка: [6.17736517819997, 0.8268307307320373, 0.8147423302743678]

---------
Индекс U0: 94; U0: 0.11770000000000085
Начальные условия: [6.17736517819997, 0.8268307307

Спектр: [-0.3102546265701387, -8.104819997909198, -8.104597728122345]
Последняя точка: [3.474201011604181, 0.9012787807099564, 0.9441791026385588]

---------
Индекс U0: 121; U0: 0.08800000000000074
Начальные условия: [3.474201011604181, 0.9012787807099564, 0.9441791026385588]
Спектр: [-0.3103141752966599, -8.312415017273906, -8.310627562631097]
Последняя точка: [3.4085997071634324, 0.9032066207636376, 0.9458348692058353]

---------
Индекс U0: 122; U0: 0.08690000000000074
Начальные условия: [3.4085997071634324, 0.9032066207636376, 0.9458348692058353]
Спектр: [-0.31037310959033987, -8.524579293345784, -8.51782405417564]
Последняя точка: [3.345963522083875, 0.9050571120437956, 0.9473704384255439]

---------
Индекс U0: 123; U0: 0.08580000000000074
Начальные условия: [3.345963522083875, 0.9050571120437956, 0.9473704384255439]
Спектр: [-0.3104312774497559, -8.735300986321187, -8.731020923029583]
Последняя точка: [3.2861594934988245, 0.9068333142457501, 0.9487963084697817]

---------
Индекс U


---------
Индекс U0: 150; U0: 0.056100000000000635
Начальные условия: [2.3538327736697666, 0.9361501956480804, 0.9666446281464433]
Спектр: [-0.3116109262806428, -14.311870041167644, -14.31086827130162]
Последняя точка: [2.3321370509508372, 0.9368815629041604, 0.9669758815323788]

---------
Индекс U0: 151; U0: 0.05500000000000063
Начальные условия: [2.3321370509508372, 0.9368815629041604, 0.9669758815323788]
Спектр: [-0.31164256073826796, -14.499635660475382, -14.498657208619898]
Последняя точка: [2.311045529825622, 0.9375954197372419, 0.9672947791131905]

---------
Индекс U0: 152; U0: 0.05390000000000063
Начальные условия: [2.311045529825622, 0.9375954197372419, 0.9672947791131905]
Спектр: [-0.3116735875910967, -14.686141771741381, -14.684679084189819]
Последняя точка: [2.290530649536336, 0.9382925205591077, 0.9676020270276091]

---------
Индекс U0: 153; U0: 0.052800000000000624
Начальные условия: [2.290530649536336, 0.9382925205591077, 0.9676020270276091]
Спектр: [-0.3117040285682315

Спектр: [-0.31234334349909726, -16.96975195304374, -21.439714595003945]
Последняя точка: [1.8868778347503794, 0.9526596600813548, 0.9731003317570805]

---------
Индекс U0: 180; U0: 0.02310000000000055
Начальные условия: [1.8868778347503794, 0.9526596600813548, 0.9731003317570805]
Спектр: [-0.3123636091497129, -16.63383062104748, -22.07741247658516]
Последняя точка: [1.875799518722839, 0.9530744404163638, 0.973237580977382]

---------
Индекс U0: 181; U0: 0.02200000000000055
Начальные условия: [1.875799518722839, 0.9530744404163638, 0.973237580977382]
Спектр: [-0.3123836381860298, -16.367532872698437, -22.643449181503374]
Последняя точка: [1.8649157005416035, 0.9534831837304127, 0.9733717607210208]

---------
Индекс U0: 182; U0: 0.02090000000000055
Начальные условия: [1.8649157005416035, 0.9534831837304127, 0.9733717607210208]
Спектр: [-0.31240343721682984, -16.145724559103936, -23.16298810693893]
Последняя точка: [1.8542205812107126, 0.9538860583106961, 0.9735029806295816]

---------
Ин

Последняя точка: [1.6279049863843031, 0.9627251200426336, 0.9761386794762469]

---------
Индекс U0: 209; U0: -0.008799999999999458
Начальные условия: [1.6279049863843031, 0.9627251200426336, 0.9761386794762469]
Спектр: [-0.3128699537776308, -14.033307431946055, -32.64810493801888]
Последняя точка: [1.6207651119244906, 0.9630147909068613, 0.9762176369881795]

---------
Индекс U0: 210; U0: -0.009899999999999458
Начальные условия: [1.6207651119244906, 0.9630147909068613, 0.9762176369881795]
Спектр: [-0.31288526084816365, -13.999787567109967, -32.933006440483645]
Последняя точка: [1.6137168659271854, 0.9633014627428184, 0.9762953404423245]

---------
Индекс U0: 211; U0: -0.010999999999999458
Начальные условия: [1.6137168659271854, 0.9633014627428184, 0.9762953404423245]
Спектр: [-0.3129004580885454, -13.967503956789384, -33.21533287419818]
Последняя точка: [1.6067582401987734, 0.963585196353132, 0.9763718221220201]

---------
Индекс U0: 212; U0: -0.012099999999999458
Начальные условия: [1.


---------
Индекс U0: 238; U0: -0.04069999999999949
Начальные условия: [1.4519403870359275, 0.9700943219327319, 0.978015204819045]
Спектр: [-0.31327721839425243, -13.39284389736581, -40.05401103229736]
Последняя точка: [1.4468221082835322, 0.9703164605068367, 0.9780676920827202]

---------
Индекс U0: 239; U0: -0.04179999999999949
Начальные условия: [1.4468221082835322, 0.9703164605068367, 0.9780676920827202]
Спектр: [-0.3132901757737191, -13.378655850316628, -40.29961511910681]
Последняя точка: [1.4417556042620792, 0.9705368254415169, 0.9781195359027014]

---------
Индекс U0: 240; U0: -0.042899999999999494
Начальные условия: [1.4417556042620792, 0.9705368254415169, 0.9781195359027014]
Спектр: [-0.31330307680482117, -13.364799166772025, -40.53831606286311]
Последняя точка: [1.4367399809708035, 0.9707554447961441, 0.978170749157884]

---------
Индекс U0: 241; U0: -0.0439999999999995
Начальные условия: [1.4367399809708035, 0.9707554447961441, 0.978170749157884]
Спектр: [-0.313315922794128

Спектр: [-0.31363438278389416, -13.083515623851559, -43.10641699893996]
Последняя точка: [1.3177984518589587, 0.9760854107869114, 0.9793539924155913]

---------
Индекс U0: 268; U0: -0.0736999999999996
Начальные условия: [1.3177984518589587, 0.9760854107869114, 0.9793539924155913]
Спектр: [-0.313646163570297, -13.075657617933244, -44.2579344374898]
Последняя точка: [1.3139175093308988, 0.9762643488748561, 0.979391616319821]

---------
Индекс U0: 269; U0: -0.0747999999999996
Начальные условия: [1.3139175093308988, 0.9762643488748561, 0.979391616319821]
Спектр: [-0.31365791838767837, -13.067936274537725, -42.98922135951107]
Последняя точка: [1.3100688142212726, 0.976442135119702, 0.9794288677704943]

---------
Индекс U0: 270; U0: -0.0758999999999996
Начальные условия: [1.3100688142212726, 0.976442135119702, 0.9794288677704943]
Спектр: [-0.313669648084102, -13.060348062105197, -42.37771879003831]
Последняя точка: [1.306251910208173, 0.9766187843237363, 0.9794657528135493]

---------
Индекс

In [ ]:
# INHERITANCE RIGHT-LEFT

for (idx_Px, Px) in enumerate(Pxrange)
    
    if idx_Px == 1 # Проверка на первую итерацию
        global u0_lc = u0
    end
    
    outputPx(idx_Px ,Px, u0_lc)
    
    ds = init_ds_Px(p, Px, Pxindex, u0_lc) # Инициализация системы
    u0_lc = goto_attractor(ds, time_attract, tt) # Выход на аттрактор
    ds = init_ds_Px(p, Px, Pxindex, u0_lc)
    ΛΛ = spectrum(ds) # Вычисление спектра
    
    outputIter(ΛΛ, u0_lc)
    
    save_output_Px(idx_Px, ΛΛ, u0_lc) # Сохранение значений
    
    separate()
    #save_tofile()
end

In [ ]:
for (idx_Px, Px) in enumerate(Pxrange)
    for (idx_Py, Py) in enumerate(Pyrange)
    
        if idx_Py == 1 # Если использовать while условие не нужно!
            continue
        end
        
        u0_lc = u0s[idx_Px, idx_Py- 1, :]
        
        output(idx_Px, idx_Py, Px, Py, u0_lc)
        
        ds = init_ds(p, Pxindex, Pyindex, Px, Py, u0)
        u0_lc = goto_attractor(ds, time_attract, tt) # Выход на аттрактор
        ds = init_ds(p, Pxindex, Pyindex, Px, Py, u0)
        ΛΛ = spectrum(ds) # Вычисление спектра
        
        outputIter(ΛΛ, u0_lc)
        
        save_output(idx_Px, idx_Py, ΛΛ, u0_lc)
        
        separate()
    end
    # save_tofile()
end